# Create Random Fish Dataset
This notebook creates the mythical fish #r1 and #r2 by taking random neurons from other fish and coding them with random stimuli for a time interval of 2000 time points.

In [1]:
import numpy as np
import pandas as pd
import random
import math
from util_functions import load_file, pickle_file, starting_run, finished_run

### Notebook Parameters:

In [2]:
random.seed(10)
data_raw_pickled_df = load_file('data_meta_pickled/data_raw_pickled_df.pickle')
data_dir = 'data_raw_pickled/'

#load all fish neuron data into dict for easy retrieval
neuron_data_list = []
for row in data_raw_pickled_df.itertuples():
    neuron_data_list.append(load_file(row.rel_path))

### Notebook Functions

In [3]:
def create_cell_XYZ():
    # note that for convenience, this was just copied from
    # the first fish (to allow for a fish-like shape of the
    # data set).
    return load_file(data_raw_pickled_df.iat[0, 2].replace('CellRespZ', 'mat_dict'))["cell_XYZ"]

In [4]:
def generate_fish_stimuli():
    stim_list = []
    while(len(stim_list) < 3878):
        stim_number = random.randint(0,23)
        stim_duration = random.randint(1,100 if (3878-len(stim_list)) > 100 else (3878-len(stim_list)))
        stim_list = np.append(stim_list, np.full(stim_duration, stim_number))
    return stim_list.astype(np.int)

In [5]:
def generate_fish_data(num_cells):
    ret_list = []
    cells_per_fish = num_cells / len(neuron_data_list)
    for fish in neuron_data_list:
        random.shuffle(fish)
        neuron_data = fish[:, 0:math.floor(cells_per_fish)]
        #to ensure that the neuron is 3878 time points long (the average among all real fish)
        while(len(neuron_data) < 3878):
            neuron_data = np.append(neuron_data, neuron_data, axis=0)
        neuron_data = neuron_data[0:3878]
        if len(ret_list) == 0:
            ret_list=neuron_data
        else:
            ret_list = np.append(ret_list, neuron_data, axis=1)
        random.shuffle(ret_list)
        for row in ret_list:
            random.shuffle(row)
    return ret_list

### Main Code Execution

In [6]:
starting_run()
for fish in ["subject_r1", "subject_r2"]:
    # Create fish mat dict
    mat_dict = {}
    mat_dict["cell_XYZ"] = create_cell_XYZ()
    mat_dict["stim_full"] = generate_fish_stimuli()
    pickle_file(data_dir + fish + '_mat_dict.pickle', mat_dict)
    
    #Create fish neuron data
    neuron_data = generate_fish_data(len(mat_dict["cell_XYZ"]))
    save_path = data_dir + fish + '_CellRespZ.pickle'
    pickle_file(save_path, neuron_data)
    data_raw_pickled_df.loc[fish] = [fish, None, save_path, neuron_data.shape, None]
pickle_file('data_meta_pickled/data_raw_pickled_df.pickle', data_raw_pickled_df)
finished_run()

Starting  19:48:26.286981
Finished  21:04:32.921298


In [7]:
pickle_file('data_meta_pickled/data_raw_pickled_df.pickle', data_raw_pickled_df)